In [19]:
# Going to be taking the data for yearly stats from basketball reference.com
import requests
from bs4 import BeautifulSoup
years_to_grab = range(2010,2019)

html_format = 'https://www.basketball-reference.com/leagues/NBA_2012_per_game.html'

# iterate through each year
all_soups = {}
for curr_year in years_to_grab:
    # to iterate through each year just replace the year from above
    curr_html = html_format.replace("2012", str(curr_year))
    # get data from proper page  and store in all_soups
    page = requests.get(curr_html)
    all_soups[str(curr_year)] = BeautifulSoup(page.text,'lxml')

In [20]:
# For each year lets extract the data and respective headers
all_data = {}
all_headers = {}
for curr_year in years_to_grab:
    data = []
    headers = []
    table = all_soups[str(curr_year)].find('table')
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    nRows = len(rows)
    print nRows

    # to actually extract the data for each year
    for curr_row in rows:
        all_col_that_row = curr_row.find_all('td')
        if len(all_col_that_row) > 0:
            data_curr_row = []
            for curr_col in all_col_that_row:
                # if isnumeric convert to float for later math
                if isfloat(curr_col.text):
                    data_curr_row.append(float(curr_col.text))
                else:
                    data_curr_row.append(curr_col.text)      
            data.append(data_curr_row)
    # now lets save
    all_data[str(curr_year)] = data
    

    # lets also get the header information so can index into each category quickly
    table_head = table.find('thead')
    table_head_row = table_head.find('tr')
    th_col = table_head_row.find_all('th')
    for curr_head in th_col:
        headers.append(curr_head.text)

    # have to remove first value or headers bc rank isn't an actual value in data scraped
    headers.remove('Rk')
    all_headers[str(curr_year)] = headers


600
647
574
596
635
675
601
619
690


In [73]:
# all helper functions for this script
import numpy as np

# quickly test if value is a float
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

# make a function that quickly will give you all values in a column that you care about
# assumes that the length of the headers label matches the number of columns in the dataset
def get_stats_player(dataset,headers,year,stat="all",name="all",pos='all'):
    if isinstance(year,str):
        curr_data = dataset[year]
        curr_headers = headers[year]
    else:
        curr_data = dataset[str(year)]
        curr_headers = headers[str(year)]
    # if want all stats see what that should be
    if stat == "all":
        stat = [curr_headers[i] for i in range(0,len(curr_data[0])) if isfloat(curr_data[0][i])]
    # check if type of data is a list and if not make it one
    if isinstance(stat, list) == False:
        stat = [stat]
    # these are the columns to use based off the stats selected
    col_to_use = []
    for curr_category in stat:
        col_to_use.append(curr_headers.index(curr_category))
    # for indexing purposes lets grab all of the names and all of the positions
    all_names =[]
    all_pos = []
    for curr_row in range(0,np.shape(curr_data)[0]):
        # get all the names
        all_names.append(curr_data[curr_row][curr_headers.index("Player")])
        # get all the positions
        all_pos.append(curr_data[curr_row][curr_headers.index("Pos")])
#     print all_pos
    # now grab that col of the dataset for those players you want
    data_for_relevant_col = []
    for curr_col in col_to_use:
        curr_vec = []
        for curr_row in range(0,np.shape(curr_data)[0]):
            curr_vec.append(curr_data[curr_row][curr_col])
        data_for_relevant_col.append(curr_vec)
    # assuming you will either sort by name or position
    if name!= "all":
        final_output = []
        for curr_cat in range(0,np.shape(data_for_relevant_col)[0]):
            final_output.append(data_for_relevant_col[curr_cat][all_names.index(name)])
    elif pos!= "all":
        # this is the index of all rows with this position
        idx = [curr_pos for curr_pos in range(0,len(all_pos)) if pos in all_pos[curr_pos]]
        final_output = []
        for curr_cat in range(0,np.shape(data_for_relevant_col)[0]):
            for curr_idx in idx:
                final_output.append(data_for_relevant_col[curr_cat][curr_idx])  
    else:
        final_output = output_data
    return final_output


In [78]:
# test = get_stats_player(all_data,all_headers,2018,stat="all",name="Alex Abrines")
test = get_stats_player(all_data,all_headers,2018,stat="Player",pos="PG")
print test
print headers


[u'Kadeem Allen', u'Ryan Arcidiacono', u'D.J. Augustin', u'Wade Baldwin', u'Lonzo Ball', u'J.J. Barea', u'Eric Bledsoe', u'Eric Bledsoe', u'Eric Bledsoe', u'Malcolm Brogdon', u'Aaron Brooks', u'Bobby Brown', u'Lorenzo Brown', u'Trey Burke', u'Dwight Buycks', u'Jose Calderon', u'Isaiah Canaan', u'Isaiah Canaan', u'Michael Carter-Williams', u'Alex Caruso', u'Mario Chalmers', u'Darren Collison', u'Mike Conley', u'Quinn Cook', u'Stephen Curry', u'Malcolm Delaney', u'Matthew Dellavedova', u'Spencer Dinwiddie', u'PJ Dozier', u'Goran Dragic', u'Larry Drew', u'Larry Drew', u'Larry Drew', u'Kris Dunn', u'Tyler Ennis', u'Jawun Evans', u'Tyreke Evans', u'Dante Exum', u'Kay Felder', u'Kay Felder', u'Kay Felder', u'Raymond Felton', u"De'Aaron Fox", u'Tim Frazier', u'Jonathan Gibson', u'Jerian Grant', u'Josh Gray', u'Devin Harris', u'Devin Harris', u'Devin Harris', u'Andrew Harrison', u'Shaquille Harrison', u'George Hill', u'George Hill', u'George Hill', u'Kyrie Irving', u'Jarrett Jack', u'Aaron Jac

In [23]:
# for saving
import json
  
json = json.dumps(all_data)
f = open("/Users/rohanramesh/Documents/SportsData/NBA/YearlyData.json","w")
f.write(json)
f.close()

In [48]:
matching = [curr_pos for curr_pos in range(0,len(test)) if 'PF' in test[curr_pos]]
print matching

[1, 10, 13, 15, 16, 20, 34, 42, 48, 49, 54, 64, 66, 67, 68, 69, 70, 110, 114, 123, 127, 132, 148, 152, 157, 169, 171, 175, 186, 195, 208, 212, 215, 219, 223, 225, 226, 227, 228, 229, 240, 241, 242, 256, 261, 281, 283, 284, 286, 287, 288, 293, 303, 310, 312, 313, 314, 316, 320, 335, 344, 346, 349, 351, 357, 364, 378, 379, 387, 388, 392, 393, 398, 407, 410, 416, 417, 418, 419, 428, 430, 432, 433, 435, 456, 464, 467, 473, 482, 485, 489, 499, 506, 507, 510, 515, 517, 542, 545, 552, 563, 573, 578, 586, 590, 592, 604, 605, 606, 617, 628, 632, 634, 636, 638, 643, 645, 648, 649, 650, 652, 656]
